In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        
        # Input: (3, 200, 200)
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            padding=1  # keeps spatial size before pooling
        )
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # After Conv + Pooling:
        # 200x200 -> Conv(3x3 same padding) → 200x200
        # 200x200 -> MaxPool2d(2x2) → 100x100
        # So tensor shape becomes: (32, 100, 100)
        self.flatten_dim = 32 * 100 * 100
        
        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        
        self.sigmoid = nn.Sigmoid()  # output for binary classification

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = x.view(x.size(0), -1)  # flatten
        
        x = F.relu(self.fc1(x))
        
        #x = self.sigmoid(self.fc2(x))
        x = self.fc2(x)
        
        return x

# Instantiate model
model = MyCNN()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

print(model)


MyCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320000, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


# Question 1

Which loss function you will use?

- nn.MSELoss()
- nn.BCEWithLogitsLoss()
- nn.CrossEntropyLoss()
- nn.CosineEmbeddingLoss()
(Multiple answered can be correct, so pick any)

Answer: nn.BCEWithLogitsLoss()

In [3]:
criterion = nn.BCEWithLogitsLoss()

# Question 2

What's the total number of parameters of the model? You can use torchsummary or count manually.

- 896
- 11214912
- 15896912
- 20073473

In [4]:
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
         MaxPool2d-2         [-1, 32, 100, 100]               0
            Linear-3                   [-1, 64]      20,480,064
            Linear-4                    [-1, 1]              65
Total params: 20,481,025
Trainable params: 20,481,025
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 12.21
Params size (MB): 78.13
Estimated Total Size (MB): 90.79
----------------------------------------------------------------


Answer: 20,481,025 (d)

In [17]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip()
])
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [18]:
# ---- Paths to your data ----
train_dir = "data/train"
test_dir = "data/test"

# ---- Create dataset objects ----
train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=train_transforms
)

validation_dataset = datasets.ImageFolder(
    root=test_dir,
    transform=test_transforms
)

# ---- Create DataLoaders ----
train_loader = DataLoader(
    train_dataset,
    batch_size=20,
    shuffle=True
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=20,
    shuffle=False
)

print("Train samples:", len(train_dataset))
print("Test samples:", len(validation_dataset))
print("Classes:", train_dataset.classes)


Train samples: 800
Test samples: 201
Classes: ['curly', 'straight']


In [13]:
# Select GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move model to device
model.to(device)

Using device: cpu


MyCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320000, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [19]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.7348, Acc: 0.6050, Val Loss: 0.6205, Val Acc: 0.7114
Epoch 2/10, Loss: 0.5957, Acc: 0.6863, Val Loss: 0.5998, Val Acc: 0.7065
Epoch 3/10, Loss: 0.5777, Acc: 0.6975, Val Loss: 0.5821, Val Acc: 0.7015
Epoch 4/10, Loss: 0.5630, Acc: 0.7000, Val Loss: 0.6790, Val Acc: 0.7214
Epoch 5/10, Loss: 0.5397, Acc: 0.7063, Val Loss: 0.5979, Val Acc: 0.7264
Epoch 6/10, Loss: 0.5301, Acc: 0.7425, Val Loss: 0.5909, Val Acc: 0.7264
Epoch 7/10, Loss: 0.5276, Acc: 0.7450, Val Loss: 0.6132, Val Acc: 0.7264
Epoch 8/10, Loss: 0.4894, Acc: 0.7312, Val Loss: 0.6141, Val Acc: 0.7114
Epoch 9/10, Loss: 0.5091, Acc: 0.7312, Val Loss: 0.5602, Val Acc: 0.7264
Epoch 10/10, Loss: 0.4895, Acc: 0.7588, Val Loss: 0.5560, Val Acc: 0.7363


# Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.05
- 0.12
- 0.40
- 0.84

Answer: d

# Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.007
- 0.078
- 0.171
- 1.710

Answer: 0.149163640192761 (c)

# Question 5

Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model. we want to continue training the model we already started training.
What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.008
- 0.08
- 0.88
- 8.88

Answer: (c)

# Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.08
- 0.28
- 0.68
- 0.98

Answer: 0.7254 (c)